In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

from utils import normalize

In [2]:
path = 'net_salary_per_town_categories.csv'

df = pd.read_csv(path)

df.columns = [normalize(c) for c in df.columns]

In [3]:
df.head()

,codgeo,libgeo,snhm14,snhmc14,snhmp14,snhme14,snhmo14,snhmf14,snhmfc14,snhmfp14,snhmfe14,snhmfo14,snhmh14,snhmhc14,snhmhp14,snhmhe14,snhmho14,snhm1814,snhm2614,snhm5014,snhmf1814,snhmf2614,snhmf5014,snhmh1814,snhmh2614,snhmh5014
0,01004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,9.6,15.0,26.4,16.7,11.0,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,10.0,14.7,23.3,15.8,11.3,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,9.5,15.3,30.2,17.2,12.4,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,9.8,13.8,24.1,14.4,11.7,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codgeo     5136 non-null   object 
 1   libgeo     5136 non-null   object 
 2   snhm14     5136 non-null   float64
 3   snhmc14    5136 non-null   float64
 4   snhmp14    5136 non-null   float64
 5   snhme14    5136 non-null   float64
 6   snhmo14    5136 non-null   float64
 7   snhmf14    5136 non-null   float64
 8   snhmfc14   5136 non-null   float64
 9   snhmfp14   5136 non-null   float64
 10  snhmfe14   5136 non-null   float64
 11  snhmfo14   5136 non-null   float64
 12  snhmh14    5136 non-null   float64
 13  snhmhc14   5136 non-null   float64
 14  snhmhp14   5136 non-null   float64
 15  snhmhe14   5136 non-null   float64
 16  snhmho14   5136 non-null   float64
 17  snhm1814   5136 non-null   float64
 18  snhm2614   5136 non-null   float64
 19  snhm5014   5136 non-null   float64
 20  snhmf181

## Exclusion de la Corse

In [5]:
print(df.shape)
df = df[df['codgeo'].astype(str).apply(lambda x: ('A' not in x) and ('B' not in x))]
print(df.shape)

(5136, 26)
(5107, 26)


In [6]:
df.loc[:, 'codgeo'] = df.loc[:, 'codgeo'].astype(int)
df['numero_departement'] = df['codgeo'] // 1000

## Exclusion des DOM TOM

In [7]:
print(df.shape)
df = df[df['codgeo'] < 96000]
print(df.shape)

df.head()

(5107, 27)
(5023, 27)


,codgeo,libgeo,snhm14,snhmc14,snhmp14,snhme14,snhmo14,snhmf14,snhmfc14,snhmfp14,snhmfe14,snhmfo14,snhmh14,snhmhc14,snhmhp14,snhmhe14,snhmho14,snhm1814,snhm2614,snhm5014,snhmf1814,snhmf2614,snhmf5014,snhmh1814,snhmh2614,snhmh5014,numero_departement
0,1004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,9.6,15.0,26.4,16.7,11.0,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6,1
1,1007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,10.0,14.7,23.3,15.8,11.3,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4,1
2,1014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,9.5,15.3,30.2,17.2,12.4,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6,1
3,1024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9,1
4,1025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,9.8,13.8,24.1,14.4,11.7,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9,1


## Gestion des villes avec arrondissement

In [8]:
df[df['numero_departement'] == 75 ]

,codgeo,libgeo,snhm14,snhmc14,snhmp14,snhme14,snhmo14,snhmf14,snhmfc14,snhmfp14,snhmfe14,snhmfo14,snhmh14,snhmhc14,snhmhp14,snhmhe14,snhmho14,snhm1814,snhm2614,snhm5014,snhmf1814,snhmf2614,snhmf5014,snhmh1814,snhmh2614,snhmh5014,numero_departement
3923,75056,Paris,22.2,31.9,17.2,12.3,13.2,19.1,26.7,15.8,12.1,12.5,24.7,35.8,18.8,12.4,13.3,11.8,21.7,28.3,11.4,19.5,21.8,12.2,23.5,33.8,75


In [9]:
df[df['libgeo'].isin(['Paris', 'Lyon', 'Marseille', 'Nice'])]


,codgeo,libgeo,snhm14,snhmc14,snhmp14,snhme14,snhmo14,snhmf14,snhmfc14,snhmfp14,snhmfe14,snhmfo14,snhmh14,snhmhc14,snhmhp14,snhmhe14,snhmho14,snhm1814,snhm2614,snhm5014,snhmf1814,snhmf2614,snhmf5014,snhmh1814,snhmh2614,snhmh5014,numero_departement
186,6088,Nice,13.2,23.5,14.5,10.5,11.0,12.1,20.9,13.7,10.4,9.2,14.1,25.1,15.2,10.8,11.4,9.7,13.1,15.4,9.3,12.1,13.6,9.9,13.8,17.0,6
397,13055,Marseille,13.8,23.8,14.6,10.4,10.8,12.5,20.9,13.5,10.3,8.9,14.8,25.5,15.8,10.7,11.1,10.0,13.6,16.1,9.3,12.6,14.0,10.6,14.3,17.8,13
3609,69123,Lyon,15.9,24.5,14.3,10.9,11.2,14.0,21.4,13.6,10.7,9.7,17.4,26.2,15.1,11.3,11.5,10.2,15.8,20.4,9.8,14.4,16.2,10.6,16.9,23.8,69
3923,75056,Paris,22.2,31.9,17.2,12.3,13.2,19.1,26.7,15.8,12.1,12.5,24.7,35.8,18.8,12.4,13.3,11.8,21.7,28.3,11.4,19.5,21.8,12.2,23.5,33.8,75


In [10]:
df.loc[df['libgeo'] == 'Paris', 'codgeo'] = 75000
df.loc[df['libgeo'] == 'Marseille', 'codgeo'] = 13000
df.loc[df['libgeo'] == 'Lyon', 'codgeo'] = 69000

In [11]:
df.head()

,codgeo,libgeo,snhm14,snhmc14,snhmp14,snhme14,snhmo14,snhmf14,snhmfc14,snhmfp14,snhmfe14,snhmfo14,snhmh14,snhmhc14,snhmhp14,snhmhe14,snhmho14,snhm1814,snhm2614,snhm5014,snhmf1814,snhmf2614,snhmf5014,snhmh1814,snhmh2614,snhmh5014,numero_departement
0,1004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,9.6,15.0,26.4,16.7,11.0,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6,1
1,1007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,10.0,14.7,23.3,15.8,11.3,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4,1
2,1014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,9.5,15.3,30.2,17.2,12.4,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6,1
3,1024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9,1
4,1025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,9.8,13.8,24.1,14.4,11.7,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9,1


## Renommage des colonnes

In [12]:
df.columns

Index(['codgeo', 'libgeo', 'snhm14', 'snhmc14', 'snhmp14', 'snhme14',
       'snhmo14', 'snhmf14', 'snhmfc14', 'snhmfp14', 'snhmfe14', 'snhmfo14',
       'snhmh14', 'snhmhc14', 'snhmhp14', 'snhmhe14', 'snhmho14', 'snhm1814',
       'snhm2614', 'snhm5014', 'snhmf1814', 'snhmf2614', 'snhmf5014',
       'snhmh1814', 'snhmh2614', 'snhmh5014', 'numero_departement'],
      dtype='object')

In [13]:
new_cols = {
    'snhm14': 'salaire_moyen',
    'snhmc14': 'salaire_moyen_cad',
    'snhmp14': 'salaire_moyen_prof_int',
    'snhme14': 'salaire_moyen_emp',
    'snhmo14': 'salaire_moyen_ouv',
    'snhmf14': 'salaire_moyen_fem',
    'snhmfc14': 'salaire_moyen_fem_cad',
    'snhmfp14': 'salaire_moyen_fem_prof_int',
    'snhmfe14': 'salaire_moyen_fem_emp',
    'snhmfo14': 'salaire_moyen_fem_ouv',
    'snhmh14': 'salaire_moyen_hom',
    'snhmhc14': 'salaire_moyen_hom_cad',
    'snhmhp14': 'salaire_moyen_hom_prof_int',
    'snhmhe14': 'salaire_moyen_hom_emp',
    'snhmho14': 'salaire_moyen_hom_ouv',
    'snhm1814': 'salaire_moyen_18-26',
    'snhm2614': 'salaire_moyen_26-50',
    'snhm5014': 'salaire_moyen_50+',
    'snhmf1814': 'salaire_moyen_fem_18-26',
    'snhmf2614': 'salaire_moyen_fem_26-50',
    'snhmf5014': 'salaire_moyen_fem_50+',
    'snhmh1814': 'salaire_moyen_hom_18-26',
    'snhmh2614': 'salaire_moyen_hom_26-50',
    'snhmh5014': 'salaire_moyen_hom_50+',
}

df = df.rename(
    columns=new_cols
)
df.head()

,codgeo,libgeo,salaire_moyen,salaire_moyen_cad,salaire_moyen_prof_int,salaire_moyen_emp,salaire_moyen_ouv,salaire_moyen_fem,salaire_moyen_fem_cad,salaire_moyen_fem_prof_int,salaire_moyen_fem_emp,salaire_moyen_fem_ouv,salaire_moyen_hom,salaire_moyen_hom_cad,salaire_moyen_hom_prof_int,salaire_moyen_hom_emp,salaire_moyen_hom_ouv,salaire_moyen_18-26,salaire_moyen_26-50,salaire_moyen_50+,salaire_moyen_fem_18-26,salaire_moyen_fem_26-50,salaire_moyen_fem_50+,salaire_moyen_hom_18-26,salaire_moyen_hom_26-50,salaire_moyen_hom_50+,numero_departement
0,1004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,9.6,15.0,26.4,16.7,11.0,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6,1
1,1007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,10.0,14.7,23.3,15.8,11.3,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4,1
2,1014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,9.5,15.3,30.2,17.2,12.4,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6,1
3,1024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9,1
4,1025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,9.8,13.8,24.1,14.4,11.7,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9,1


In [14]:
df.to_csv('clean_data/salary_clean.csv', index=False)